In [ ]:
import cv2
import numpy as np
body_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap=cv2.VideoCapture(0)
data=[]
ix=0
while (1):
    ret,img = cap.read()
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = body_cascade.detectMultiScale(gray,1.3,5)
    for(x,y,w,h) in faces:
        face_component = img[y:y+h, x:x+w, :]
        fc = cv2.resize(face_component, (50, 50))
        if ix%10 == 0 and len(data) < 20:
            data.append(fc)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    ix+=1 
    cv2.imshow('img',img)
    if cv2.waitKey(1) == 27 or len(data) >= 20:
        break
cap.release()
cv2.destroyAllWindows()
data=np.asarray(data)
print(data.shape)
np.save('face_01',data)

In [ ]:
import cv2
import numpy as np
body_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap=cv2.VideoCapture(0)
data=[]
ix=0
while (1):
    ret,img = cap.read()
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = body_cascade.detectMultiScale(gray,1.3,5)
    for(x,y,w,h) in faces:
        face_component = img[y:y+h, x:x+w, :]
        fc = cv2.resize(face_component, (50, 50))
        if ix%10 == 0 and len(data) < 20:
            data.append(fc)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    ix+=1 
    cv2.imshow('img',img)
    if cv2.waitKey(1) == 27 or len(data) >= 20:
        break
cap.release()
cv2.destroyAllWindows()
data=np.asarray(data)
print(data.shape)
np.save('face_02',data)

In [1]:
import numpy as np
import cv2

# instantiate the camera object and haar cascade
cam = cv2.VideoCapture(0)
face_cas = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# declare the type of font to be used on output window
font = cv2.FONT_HERSHEY_SIMPLEX

# load the data from the numpy matrices and convert to linear vectors
f_01 = np.load('face_01.npy').reshape((20, 50*50*3))    
f_02=  np.load('face_02.npy').reshape((20, 50*50*3))   

print(f_01.shape, f_02.shape)

# create a look-up dictionary
names = {
    0: 'Praseedha',
    1: 'Praseedha'

}

# create a matrix to store the labels
labels = np.zeros((40, 1))
labels[:20, :] = 0.0    
labels[20:, :] = 1.0 

# combine all info into one data array
data = np.concatenate([f_01, f_02]) # (40, 7500)
print(data.shape, labels.shape)  # (40, 1)

# the distance and knn functions we defined earlier
def distance(x1, x2):
    return np.sqrt(((x1-x2)**2).sum())

def knn(x, train, targets, k=5):
    m = train.shape[0]
    dist = []
    for ix in range(m):
        # compute distance from each point and store in dist
        dist.append(distance(x, train[ix]))
    dist = np.asarray(dist)
    indx = np.argsort(dist)
    sorted_labels = labels[indx][:k]
    counts = np.unique(sorted_labels, return_counts=True)
    return counts[0][np.argmax(counts[1])]



(20, 7500) (20, 7500)
(40, 7500) (40, 1)


In [ ]:
from flask import Flask, render_template, Response
import cv2
import numpy as np

body_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
app = Flask(__name__)

camera = cv2.VideoCapture(0)  # use 0 for web camera
#  for cctv camera use rtsp://username:password@ip_address:554/user=username_password='password'_channel=channel_number_stream=0.sdp' instead of camera

def gen_frames():  # generate frame by frame from camera
    while True:
        # Capture frame-by-frame
        success, frame = camera.read()  # read the camera frame
        if not success:
            break
        else:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cas.detectMultiScale(gray, 1.3, 5)

            # for each face
            for (x, y, w, h) in faces:
                face_component = frame[y:y+h, x:x+w, :]
                fc = cv2.resize(face_component, (50, 50))


                lab = knn(fc.flatten(), data, labels)
           
                text = names[int(lab)]

           
                cv2.putText(frame, text, (x, y), font, 1, (255, 255, 0), 2)

                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')  # concat frame one by one and show result


@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(gen_frames(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')


@app.route('/')
def index():
    """Video streaming home page."""
    return render_template('check.html')
   


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Sep/2020 13:03:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Sep/2020 13:03:25] "GET /video_feed HTTP/1.1" 200 -
